<a href="https://colab.research.google.com/github/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/blob/main/Chapter13/Detectron2_Chapter13_DeployingToBrowser.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 13 - Deploying ONNX model to Browser
## Load a model

In [ ]:
!wget https://github.com/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/raw/main/Chapter13/onnx_model.onnx

--2023-02-17 22:20:38--  https://github.com/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/raw/main/Chapter13/onnx_model.onnx
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/main/Chapter13/onnx_model.onnx [following]
--2023-02-17 22:20:38--  https://raw.githubusercontent.com/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/main/Chapter13/onnx_model.onnx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6587 (6.4K) [application/octet-stream]
Saving to: ‘onnx_model.onnx’

onnx_model.onnx     100%[===================>]   6.43K

## Download onnx.js

In [ ]:
!wget https://cdn.jsdelivr.net/npm/onnxjs/dist/onnx.min.js

--2023-02-17 22:43:41--  https://cdn.jsdelivr.net/npm/onnxjs/dist/onnx.min.js
Resolving cdn.jsdelivr.net (cdn.jsdelivr.net)... 104.16.86.20, 104.16.89.20, 104.16.87.20, ...
Connecting to cdn.jsdelivr.net (cdn.jsdelivr.net)|104.16.86.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/javascript]
Saving to: ‘onnx.min.js’

onnx.min.js             [ <=>                ] 453.93K  --.-KB/s    in 0.04s   

2023-02-17 22:43:41 (11.0 MB/s) - ‘onnx.min.js’ saved [464829]



## Create an HTML file

In [ ]:
%%writefile load_onnx.html
<html>
  <head> </head>
  <body>
    <div id="output"></div>
    <script src="https://cdn.jsdelivr.net/npm/onnxjs/dist/onnx.min.js">
    </script>
    <script>
      const myOnnxSession = new onnx.InferenceSession();
      myOnnxSession.loadModel("./onnx_model.onnx")
      .then(() => {
        // Step 1: create an input
        const inferenceInputs = new onnx.Tensor(
          [2, 3, 4, 5], 
          'float32', [1, 4]
          );
        // Step 2: run and update output
        myOnnxSession.run([inferenceInputs]).then((output) => {
          const outputTensor = output.values().next().value;
          const msg = `model output tensor: ${outputTensor.data}`;
          document.getElementById('output').innerText = msg;
        });
      });
    </script>
  </body>
</html>


Writing load_onnx.html


## Run a web server

In [ ]:
from IPython.display import Javascript

def expose_content(port, height=200):
  display(Javascript("""
  (async ()=>{
    fm = document.createElement('iframe')
    fm.src = await google.colab.kernel.proxyPort(%s)
    fm.width = '95%%'
    fm.height = '%d'
    fm.frameBorder = 0
    document.body.append(fm)
  })();
  """ % (port, height)))

In [ ]:
get_ipython().system_raw('python3 -m http.server 8888 &') 
expose_content(8888)

<IPython.core.display.Javascript object>